data= pd.read_csv("") charger le dataset 

In [3]:
# recuper 10% du dataset pour alléger la mémoire  
data_alleger=data.sample(frac=0.1) 

In [ ]:
data_alleger.describe()

traiter les valeurs manquantes à travers la boucle for qui va parcourir tout le dataset

In [ ]:
for i in data_alleger.columns:
    print(i,'||',data_alleger[i].dtypes,'||', data_alleger[i].isna().sum(),'||','{:.2%}'.format((data_alleger.isna().sum()/len(data_alleger))[i])) 

 créer un dataframe vide dans lequel nous allons ajouter les colonnes qui nous intéressent, telles que Nom du produit, Type de données, Nombre de valeurs manquantes, Valeurs manquantes

In [ ]:
missing_data = pd.DataFrame()
missing_data['Column variable'] = data_alleger.columns
missing_data['Data type'] = list(data_alleger.dtypes)
missing_data['Missing values count'] = list(data_alleger.isna().sum())
missing_data['% Missing values'] = list((data_alleger.isna().sum()/len(data_alleger))) 
print(missing_data)

Nous visualisations les valeurs manquantes 

In [ ]:
to_plot = missing_data.sort_values(by=['% Missing values'])
fig, ax = plt.subplots(figsize=(18, 9))
ax.set_facecolor("#2E2E2E")
plt.title('Representation of the % Missing values per column', fontsize=20, y=1.03)
plt.xlabel('Columns Name', fontsize=14)
plt.ylabel('% Missing values', fontsize=14)
plt.grid(True, color="#93a1a1", alpha=0.05)
plt.xticks(rotation=90) 
plt.plot(to_plot['Column variable'],to_plot['% Missing values'],color="#E8FF41");

suppression des valeurs manquantes 

In [ ]:
data = data_alleger.dropna(axis=1, thresh= len(data_alleger)*0.75) 
for i in data.columns:
    print(i,'||',data[i].dtypes,'||', data[i].isna().sum(),'||','{:.2%}'.format((data_alleger.isna().sum()/len(data_alleger))[i])) 
    

Nous créons une nouvelle trame de données appelée missing_data2 qui contient les colonnes avec 40 % ou moins de valeurs manquantes dans leurs colonnes.

In [ ]:
missing_data2 = pd.DataFrame()
missing_data2['Column variable'] = data.columns
missing_data2['Data type'] = list(data.dtypes)
missing_data2['Missing values count'] = list(data.isna().sum())
missing_data2['% Missing values'] = list(data.isna().sum()/len(data)) 
print(missing_data2)

Rechercher des lignes entières avec des valeurs NaN

In [ ]:
miss_rows=data[data.isnull().all(axis=1)]

print('le nombre est :', len(miss_rows))

In [ ]:
drop_data = data.dropna(axis=0, how='any', subset=['product_name'], inplace=False)

missing_data3 = pd.DataFrame()
missing_data3['Column variable'] = drop_data.columns
missing_data3['Data type'] = list(drop_data.dtypes)
missing_data3['Missing values count'] = list(drop_data.isna().sum())
missing_data3['% Missing values'] = list(drop_data.isna().sum()/(len(drop_data))) 
print(missing_data3)

In [ ]:
mask = drop_data[drop_data.loc[:,['energy-kcal_100g','energy_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','proteins_100g']].isnull().sum(axis=1) == 9]

In [ ]:
drop_data_nan = drop_data.append(mask).drop_duplicates(keep=False)
drop_data_nan.info()

sauvegarder le dataset netoyer : drop_data_nan.to_csv('data_netoyer.csv',sep = '\t',index = True)